In [1]:
#importing basic libraries 
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import glob

In [2]:
#preparing and processing data
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)
train_dataset = train_datagen.flow_from_directory(
    'train_dataset',
    target_size = (64, 64),
    batch_size = 32,
    class_mode = 'categorical'
)


Found 3497 images belonging to 5 classes.


In [3]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_dataset = test_datagen.flow_from_directory('test_dataset',target_size=(64,64),batch_size=32,class_mode='categorical')


Found 820 images belonging to 5 classes.


In [7]:
#building the model and saving it 

cnn = tf.keras.models.Sequential()
#building the convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Dropout(0.5))
cnn.add(tf.keras.layers.Flatten())
#building the fully connected layer
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
#building the output layer
cnn.add(tf.keras.layers.Dense(units=5, activation='softmax'))
#compiling the model
cnn.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
cnn.fit(x=train_dataset, validation_data=test_dataset, epochs=40)

#saving the model
cnn.save('FlowerRecognition.keras',include_optimizer=True)

#laod the saved model 
#if u trained the model once u can load it again without training it just uncomment the following lines
'''
cnn = tf.keras.models.load_model("FlowerRecognition.keras")
cnn.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
'''


In [6]:
from tensorflow.keras.preprocessing import image
import glob

#process new image with our model 
for item in glob.glob("Prediction\\*"):
    flowerimage_path = item
    flowerimage = image.load_img(flowerimage_path,target_size=(64,64))
    test_image = image.img_to_array(flowerimage)
    test_image = np.expand_dims(test_image,axis=0)
    result = cnn.predict(test_image)
    flowerimage_cv = cv2.imread(flowerimage_path)
    if result[0][0] == 1:
        flower_name = 'Daisy'
    elif result[0][1] == 1:
        flower_name = 'Dandelion'
    elif result[0][2] == 1:
        flower_name = 'Rose'
    elif result[0][3] == 1:
        flower_name = 'Sunflower'
    elif result[0][4] == 1:
        flower_name = 'Tulip'
    cv2.putText(flowerimage_cv, flower_name, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    cv2.imshow('Flower', flowerimage_cv)  
    cv2.waitKey(0)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
